In [20]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

#### Devide time-series data into sequences

In [7]:
def data_split(data, split_rate = 0.7):
    train_size = int(len(data) * split_rate)
    train_data, test_data = data[:train_size], data[train_size:]
    return train_data, test_data

In [49]:
import pandas as pd
## Split the sequence
## Function to create sequences of data
def create_sequences(data, seq_length):
    sequence = []
    for i in range(len(data) - seq_length):
        one_data = (torch.tensor(data[i:i+seq_length]), data[i+seq_length])
        sequence.append(one_data)
    # print(sequence)
    return sequence

In [493]:
def build_data_loader(data, seq_length = 3, split_rate_train_test = 0.7, split_rate_train_val = 0.7, batch_size = 1, shuffle_train = False, shuffle_val = False, shuffle_test = False):
    '''
    Input the data, return training dataloader, validation dataloader, and test dataloader
    '''
    train_data, test_data = data_split(data, split_rate_train_test)
    train_data, val_data = data_split(train_data, split_rate_train_val)

    training_data_seq = create_sequences(train_data, seq_length)
    val_data_seq = create_sequences(val_data, seq_length)
    test_data_seq = create_sequences(test_data, seq_length)

    train_dataloader = DataLoader(training_data_seq, batch_size, shuffle = shuffle_train) ## Change to true later
    val_dataloader = DataLoader(val_data_seq, batch_size, shuffle = shuffle_val)
    test_dataloader = DataLoader(test_data_seq, batch_size, shuffle = shuffle_test)

    return train_dataloader, val_dataloader, test_dataloader


In [104]:
def show_data_loader(train_loader):
    '''
    a function shows every entry of train_loader
    '''
    for i, data in enumerate(train_loader):
            inputs, labels = data
            print("inputs: ", inputs, "labels: ", labels)

#### Quick split

#### Define the model

In [118]:
## Define a simple linear model
class linear_Net(nn.Module):
    '''
    A very simple linear model
    '''
    def __init__(self, seq_unit, output_size = 1):
        super().__init__()
        self.seq_unit = seq_unit
        self.fc1 = nn.Linear(seq_unit, output_size)
    
    def forward(self, x):
        x = torch.flatten(x, 1) 
        x = self.fc1(x)
        return x

In [227]:
class mlp_Net(nn.Module):
    '''
    A basic multiple linear perceptron model with a hidden layer
    '''
    def __init__(self, seq_unit, hidden_size, output_size = 1):
        super().__init__()
        self.activation = F.relu
        self.fc1 = nn.Linear(seq_unit, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.flatten(x, 1) 
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x

In [383]:
class cnn_Net(nn.Module):
    '''
    A basic convolutional neural network with a number of filters
    '''
    def __init__(self, seq_unit, kernel_size = 3, output_size = 1):
        super().__init__()
        self.seq_unit = seq_unit
        self.conv_net = nn.Sequential(
            ## vector size: 3
            nn.Conv1d(in_channels=1, out_channels= 16, kernel_size=kernel_size, stride=1, padding=1), ## 4x16
            nn.ReLU(inplace=True),
            nn.Flatten(),
            nn.Linear(seq_unit*16, 1)
        )
    
    def forward(self, x):
        ## permute to put channel in correct order
        # x = x.permute(0, 2, 1)
        out = self.conv_net(x)
        return x

In [389]:
## Define the LSTM
class lstm_Net(nn.Module):
    '''
    A long short term memory model
    '''
    def __init__(self, seq_unit, hidden_size = 32, output_size = 1):
        super().__init__()
        self.lstm = nn.LSTM(seq_unit, hidden_size, num_layers = 1, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        # x = self.fc(x[:, -1, :]) ## extract only the last time step
        return x

In [135]:
def model_loss(model, loss_function, x, y, optimizer = None):
    '''
    Apply loss function to a batch of inputs. If no optimizer is provided, 
    skip the back propagation step
    '''
    ## prediction
    # print("         Enter function model_loss:\n")
    output = model(x.float())
    # print("             output:", output)

    loss = loss_function(output, y.float())

    if optimizer is not None:
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    # print("         model_loss:", loss.item(), "\n")
    return loss.item(), len(x)

In [513]:
def train_one_epoch(model, train_dl, loss_function, device, optimizer):
    '''
    Execute 1 set of batched training within an epoch
    '''
    # print("     Enter function train_one_epoch():\n")
    ## Set the model to training mode
    model.train()
    train_losses = []
    batch_sizes = []

    ## Loop through train dataloader
    for x_train, y_train in train_dl:
        # print("train data x: ", x_train)
        # print("train data y: ", y_train)
        ## transfer the data to GPU if any
        x_train, y_train = x_train.to(device), y_train.to(device)

        ## Back propagation
        train_loss, batch_size = model_loss(model, loss_function, x_train, y_train, optimizer)

        ## Append train loss and batch size
        train_losses.append(train_loss)
        batch_sizes.append(batch_size)
    ## Calculate the average losses over all batches
    train_loss = np.sum(np.multiply(train_losses, batch_sizes)) / np.sum(batch_sizes)
    # print("batch train loss: ", train_loss)

    return train_loss

In [467]:
def val_one_epoch(model,val_dl,loss_function,device, optimizer):
    '''
    Excute 1 set of batched validation within an epoch
    '''
    # print("     Enter function val_one_epoch():\n")
    model.eval()
    with torch.no_grad():
        validation_losses = []
        batch_sizes = []

        ## Loop through the validation dataloader
        for x_valid, y_valid in val_dl:
            ## transfer the data to GPU if any
            x_valid, y_valid = x_valid.to(device), y_valid.to(device)

            ## Calculate the loss WITHOUT BACK PROPOGATION
            validation_loss, batch_size = model_loss(model, loss_function, x_valid, y_valid)

            ## Append validation loss and batch size
            validation_losses.append(validation_loss)
            batch_sizes.append(batch_size)

        ## Calculate the average losses over all batches
        val_loss = np.sum(np.multiply(validation_losses, batch_sizes)) / np.sum(batch_sizes)
        # print("batch validation loss: ", val_loss)

        return val_loss
            

In [517]:
### test the model with testing data
def test_model(model, test_dl, loss_function, optimizer):
    # Evaluate the model on the test data
    model.eval()
    with torch.no_grad():
        test_losses = []
        batch_sizes = []

        ## Loop through the validation dataloader
        for x_test, y_test in test_dl:
            # print("test data x: ", x_test)
            # print("test data y: ", y_test)

            ## Calculate the loss WITHOUT BACK PROPOGATION
            test_loss, batch_size = model_loss(model, loss_function, x_test, y_test)

            ## Append validation loss and batch size
            test_losses.append(test_loss)
            batch_sizes.append(batch_size)

        ## Calculate the average losses over all batches
        test_loss = np.sum(np.multiply(test_losses, batch_sizes)) / np.sum(batch_sizes)

        return test_loss
    print('Test Loss: {:.4f}'.format(test_loss.item()))

In [503]:
def model_fit(train_dl, val_dl, model, device, num_epochs, loss_function, optimizer):
    '''
    For num_epochs, fit the model parameters to the training data, evaluation on validation data
    For each epoch, calculate and return the training and validation loss
    '''
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        ## step 1: training
        train_loss = train_one_epoch(model, train_dl, loss_function, device, optimizer)
        train_losses.append(train_loss)

        ## step 2: validation
        val_loss = val_one_epoch(model, val_dl, loss_function, device, optimizer)
        val_losses.append(val_loss)

        print(f"Epoch{epoch} | train loss: {train_loss:.3f} | val loss: {val_loss:.3f}")
    return train_losses, val_losses

In [504]:
def model_run(train_dl, val_dl, test_dl, model, device, learning_rate = 0.001, num_epochs = 500, loss_function = None, optimizer = None):
    ## Define loss function
    if loss_function is None:
        loss_function = nn.MSELoss()
    
    ## Define optimization function
    if optimizer is None:
        optimizer = optim.Adam(model.parameters(), lr = learning_rate)

    ## run the model fit function
    train_losses, val_losses = model_fit(train_dl, val_dl, model, device, num_epochs, loss_function, optimizer)

    ## test the model
    test_losses = test_model(model, test_dl, loss_function, optimizer)

    return train_losses, val_losses, test_losses

#### Run the model

##### Read data/generate data

In [505]:
## Generate artificial data
def generate_data(num_data = 500, step = 1, func = "Linear", pattern = 1):
    '''
    Generate artificial data with different patterns 

    function 1: linear
    function 2: sin(x)

    pattern 0: return data as it is
    pattern 1: add a mask every 5 numbers
    pattern 2: add a linear function
    '''
    ## generate the index from 0 to num_data
    idx = np.arange(0, num_data) 

    ## determine the function
    if (func == "Linear"):
        initial_data = idx
    elif (func == "Sin"):
        initial_data = np.sin(idx)
    
    ## determine the patterns
    if(pattern == 0):
        return initial_data
        
    for idx in range(0, len(initial_data)):
        if(idx % 50 == 0):
            if(pattern == 1):
                if(idx - 4 >= 0): initial_data[idx] += 1
                if(idx - 3 >= 0): initial_data[idx] += 2
                if(idx - 2 >= 0): initial_data[idx] += 3
                if(idx - 1 >= 0): initial_data[idx] += 4
                if(idx >= 0 and idx < len(initial_data)): initial_data[idx] += 5
                if(idx + 1 < len(initial_data)): initial_data[idx] += 4
                if(idx + 2 < len(initial_data)): initial_data[idx] += 3
                if(idx + 3 < len(initial_data)): initial_data[idx] += 2
                if(idx + 4 < len(initial_data)): initial_data[idx] += 1
                
            elif(pattern == 2):
                initial_data[idx] = initial_data[idx]*5 + 8

    return initial_data



In [506]:
def import_data(dataset = "EVcharging_data.csv"):
    '''
    Import existing data from directory
    '''
    pass

In [507]:
def read_data(datatype = 1, num_data = 500, step = 1, func = "Linear", pattern = 1, dataset = "EVCharging_data.csv"):
    '''
    User can choose to generate data or import data
    1: generate data
    2: read data
    '''
    if(datatype == 1):
        return generate_data(num_data = 500, step = 1, func = "Linear", pattern = 1)
    else:
        return import_data(dataset = dataset)

In [508]:
sequence_length = 3
data = read_data(datatype = 1, num_data = 500, step = 1, func = "Linear", pattern = 1)

train_dataloader, val_dataloader, test_dataloader = build_data_loader(data, seq_length=sequence_length, batch_size=20)

# show_data_loader(train_dataloader)

#### Data trend visualization
##### change them into a function

In [509]:
## Change 
import altair as alt

source = pd.DataFrame({
    'x': np.arange(0,len(data)),
    'f(x)': data
  })

alt.Chart(source).mark_line().encode(
      x='x',
      y='f(x)'
  )

alt.Chart(...)

#### Model bundle

In [510]:
from torchvision import models
from torchsummary import summary

In [515]:
def model_bundle(train_dataloader, val_dataloader, test_dataloader, seq_len = sequence_length, model_type = 4, num_epoch = 1000, verbose = True):
    '''
    Model bundle function that takes "Linear", "MLP", "CNN", "LSTM" as input
    "Linear": 1
    "MLP": 2
    "CNN": 3
    "LSTM": 4
    If "verbose", output the model summary
    '''
    if(model_type == 1):
        model = linear_Net(seq_len, 1)
    elif(model_type == 2):
        model = mlp_Net(seq_len, hidden_size = 32, output_size = 1)
    elif(model_type == 3):
        model = cnn_Net(seq_len, kernel_size = 3, output_size = 1)
    elif(model_type == 4):
        model = lstm_Net(seq_len, hidden_size = 32, output_size = 1)

    if(verbose and model_type < 4):
        print("+--------------------+")
        print("|Print the linear model summary|")
        print("+--------------------+")
        print(summary(model, (1, seq_len)))
    
    ## train the model
    train_losses, val_losses, test_losses = model_run(train_dataloader, val_dataloader, test_dataloader, model, device, num_epochs = num_epoch)

    return train_losses, val_losses, test_losses

In [518]:
train_losses, val_losses, test_losses = model_bundle(train_dataloader = train_dataloader, val_dataloader=val_dataloader, test_dataloader = test_dataloader, seq_len=sequence_length, model_type=4, num_epoch=10, verbose=True)

Epoch0 | train loss: 20019.207 | val loss: 89711.287
Epoch1 | train loss: 19951.585 | val loss: 89618.014
Epoch2 | train loss: 19882.360 | val loss: 89436.156
Epoch3 | train loss: 19777.610 | val loss: 89072.037
Epoch4 | train loss: 19616.271 | val loss: 88754.042
Epoch5 | train loss: 19549.158 | val loss: 88611.432
Epoch6 | train loss: 19460.380 | val loss: 88384.397
Epoch7 | train loss: 19399.312 | val loss: 88243.109
Epoch8 | train loss: 19298.390 | val loss: 87932.287
Epoch9 | train loss: 19206.060 | val loss: 87650.205


#### Result diagnoses

#### Loss function visualization

In [448]:
def quick_loss_plot(train_losses, val_losses, model_type = 4, loss_type = "MSE Loss", sparse_n = 0):
    '''
    For each train/test loss trajectory, plot loss by epoch
    '''
    model_dict = {
        1: "Linear",
        2: "MLP",
        3: "CNN",
        4: "LSTM"
    }

    data_label_list = [(train_losses, val_losses, model_dict[model_type])]

    for i,(train_data,val_data,label) in enumerate(data_label_list):    
        plt.plot(train_data,linestyle='--',color=f"C{i}", label=f"{label} Train")
        plt.plot(val_data,color=f"C{i}", label=f"{label} Val",linewidth=3.0)

    plt.legend()
    plt.ylabel(loss_type)
    plt.xlabel("Epoch")
    plt.legend(bbox_to_anchor=(1,1),loc='upper left')
    plt.show()

In [449]:
def quick_prediction_plot(data_label_list, sparse_n = 0):
    pass

In [ ]:
### Test with testing data
with torch.no_grad():
    # shift train predictions for plotting
    train_plot = np.ones_like(data) * np.nan
    y_pred = model(X_train)
    y_pred = y_pred[:, -1, :]
    train_plot[lookback:train_size] = model(X_train)[:, -1, :]
    # shift test predictions for plotting
    test_plot = np.ones_like(data) * np.nan
    test_plot[train_size+lookback:len(data)] = model(X_test)[:, -1, :]
# plot
plt.plot(data, c='b')
plt.plot(train_plot, c='r')
plt.plot(test_plot, c='g')
plt.show()